<a href="https://colab.research.google.com/github/GNuSeekK/MachineLearning/blob/main/%EC%A3%BC%EC%8B%9D_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5_%ED%95%99%EC%8A%B5_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 드라이브 Mount

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# Frame_work import

In [3]:
!pip install cryptography
!pip install pymysql
!pip install --upgrade pandas_profiling
 
#드라이브 마운트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
import os, sys
import pymysql
import cryptography
import datetime as dt
import pandas_profiling
import requests
from bs4 import BeautifulSoup
import multiprocessing as mp
from multiprocessing import Pool, Manager

     |████████████████████████████████| 256kB 7.0MB/s 
     |████████████████████████████████| 645kB 9.8MB/s 
     |████████████████████████████████| 1.1MB 17.3MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 3.1MB 22.5MB/s 
     |████████████████████████████████| 112kB 51.0MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 10.1MB 39.6MB/s 
     |████████████████████████████████| 296kB 49.7MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27085 sha256=138e51723463ead55cfce72a6473317022b71c5488db70cd9b32ccef548d0b61
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107413 sha256=cd6cac9f759da5f7450dd17c0f04992131afd3ef1e891321ce35aa686c892850
  Stored in directory: /root/.cache/pip/wheels/c0/a3/b0/f27b1cfe32ea131a3715169132ff6d8565

# 함수 및 실행 전 기초 작업

## 1. DB 연결


### Function **db_connecting_aws**
### Function **db_connecting_local**
* 매개변수(parameter) : db_name

In [12]:
def db_connecting_aws(db_name):
    path = r'/content/gdrive/My Drive/DB 정보/AWS'
    f = open(path)
    id, pw, host = f.read().split()
    db = pymysql.connect(
        user = id,
        port = 3306,
        passwd = pw,
        host = host,
        db = db_name,
        charset = 'utf8',
        cursorclass = pymysql.cursors.DictCursor
    )
    return db
    
def db_connecting_local(db_name):
    path = r'/content/gdrive/My Drive/DB 정보/LOCAL'
    f = open(path)
    id, pw, host = f.read().split()
    db = pymysql.connect(
        user = id,
        port = 3306,
        passwd = pw,
        host = host,
        db = db_name,
        charset = 'utf8',
        cursorclass = pymysql.cursors.DictCursor
    )
    return db

### 실제 DB에 연결하기
* cpu의 갯수만큼 연결한다.
* 하나로 쓸 db, 여러개로 사용할 db_list를 따로 구성한다

In [ ]:
#db 작업
 
db = db_connecting_aws('stockdb')

db_list_aws = []
db_list_local = []
num_cores = mp.cpu_count()
for _ in range(num_cores):
    db_list_aws.append(db_connecting_aws('stockdb'))
    db_list_local.append(db_connecting_local('stockdb'))

## 2. Progress_bar 함수

### Function **my_pbar**
* 매개변수(parameter) : value, endvalue, bar_length(default = 20)

In [ ]:
def my_pbar(value, endvalue, bar_length = 20):
    if value % 100 == 0 or value == endvalue:
        percent = float(value) / endvalue
        complete = '<' + '▮' * int(round(percent * bar_length)-1)
        remain = '▯' * (bar_length - len(complete)) + '>'
        if percent <= 0.33:
            color = 91
            sys.stdout.write(f"\r\033[{color}m Percent: [{complete+remain}] {round(percent*100,2)}%\033[0m")
        elif percent <= 0.66:
            color = 93
            sys.stdout.write(f"\r\033[{color}m Percent: [{complete+remain}] {round(percent*100,2)}%\033[0m")
        elif percent == 1:
            color = 92
            sys.stdout.write(f"\r\033[{color}m Percent: [{complete+remain}] {round(percent*100,2)}%\033[0m")
        else:
            color = 96
            sys.stdout.write(f"\r\033[{color}m Percent: [{complete+remain}] {round(percent*100,2)}%\033[0m")
        sys.stdout.flush()

## 3. Manager를 통해 공유메모리 관리하기

In [14]:
num_cores = mp.cpu_count()
manager = Manager()
db_index = manager.list(list(range(num_cores)))
progress = manager.list([0 for x in range(num_cores)])

## 4. train_data 추출용 함수

### Function **input_making**
* 매개변수(parameter) : num
> 추출해낼 데이터를 num을 이용하여 랜덤으로 뽑아내기 위한 함수

In [ ]:
# primary key인 num을 이용하여 추출할 데이터 셋을 랜덤으로 뽑기 위한 함수

def input_making(num):
    global num_list
    index = db_index.pop(0)
    db = db_list_aws[index]
    sql = f"""
        select 
            C_code, 
            DATE_FORMAT(date, '%Y-%m-%d'),
            DATE_FORMAT(time, '%H:%i:%s') as time 
        from mdtbl2 
        where num = {num}
    """
 
    #start_time = dt.datetime.now()
    data = pd.read_sql(sql,db)
    #print('걸린시간 : %s' %(dt.datetime.now() - start_time))
 
    db_index.append(index)
    progress[index] = progress[index] + 1
    my_pbar(sum(progress),len(num_list))
    if len(data) > 0:
        return list(data.values[0])

### Function **data_making**
* 매개변수(parameter) : code, date, time
> input_making로 뽑아낸 데이터를 이용해 train_data를 추출하는 함수

In [ ]:
# 주어지는 데이터 = code, date, time, db, date_list, input_data
# 필요한 데이터 output 값
# 최고값 : 1분뒤, 2분뒤, 3분뒤, 5분뒤, 10분뒤, 5분간, 10분간, 20분간, 60분간, 오늘, 1일뒤, 2일뒤, 3일뒤, 5일뒤, 10일뒤, 3일간, 5일간, 10일간, 20일간

# 삼성전자를 이용하여 주식이 거래된 날짜의 list를 구한다.
sql = """
    select distinct date from mdtbl2 where C_code = '005930'
"""
date_list = pd.read_sql(sql, db)
date_list = date_list['date'].tolist()
date_list = [str(x) for x in date_list]

def data_making(code, date, time):
    global input_data
    try:
        index = db_index.pop(0)
        global date_list
        date_index = date_list.index(date)
        date_min = date_list[date_index - 30] #30일전
        date_max = date_list[date_index + 20] #20일후
        db = db_list_local[index]
        start_time = dt.datetime.now()
        sql = f"""set 
                    @price = (select priceE from mdtbl2
                                where 
                                    C_code = '{code}' and 
                                    date = '{date}' and 
                                    time = '{time}'
                            ),
                    @price_percent = (select priceE from mdtbl2
                                where 
                                    C_code = '{code}' and 
                                    date = '{date}' and 
                                    time = '{time}'
                            ) / 100,
                    @Issued_shares = (select Issued_shares from ftbl 
                                        where 
                                            C_code = '{code}' and 
                                            dateS <= '{date}' and 
                                            dateE >= '{date}'
                                    ),
                    @Issued_shares_minute = (select Issued_shares from ftbl 
                                        where 
                                            C_code = '{code}' and 
                                            dateS <= '{date}' and 
                                            dateE >= '{date}'
                                    ) / 390,
                    @BPS = (select BPS from ftbl
                                where
                                    C_code = '{code}' and
                                    dateS <= '{date}' and
                                    dateE >= '{date}' 
                        )
            """
        with db.cursor() as cursor:
            cursor.execute(sql)
            db.commit() 
        # 분봉 데이터 가져오기
        sql = f"""
            Select
                TBL1.C_code as C_code,
                (TBL1.PriceS / TBL2.PriceE - 1) * 100 as PriceS,
                (TBL1.PriceH / TBL2.PriceE - 1) * 100 as PriceH,
                (TBL1.PriceL / TBL2.PriceE - 1) * 100 as PriceL,
                (TBL1.PriceE / TBL2.PriceE - 1) * 100 as PriceE,
                TBL1.trade as Trade

                from
                (
                select
                    C_code,
                    date, 
                    time,
                    PriceS,
                    PriceH,
                    PriceL,
                    PriceE, 
                    trade / @Issued_shares_minute as trade
                    from mdtbl2
                    where
                        C_code = '{code}' and
                        date = '{date}' and
                        time > Date_SUB(time('{time}'), interval 60 minute) and
                        time <= '{time}'
                ) as TBL1,

                (
                select
                    C_code, 
                    date,
                    Date_ADD(time, interval 1 minute) as time,
                    PriceE
                    from mdtbl2
                    where
                        C_code = '{code}' and
                        (
                            (
                            date = '{date}' and
                            time > Date_SUB(time('{time}'), interval 61 minute) and
                            time < '{time}'
                            )
                            or
                            (
                            date = '{date_list[date_index - 1]}' and
                            time = '15:30:00'
                            )

                        )
                ) as TBL2
                where
                    TBL1.C_code = TBL2.C_code and
                    (
                        (
                        TBL1.date = TBL2.date and
                        TBL1.time = TBL2.time
                        )
                        or
                        (
                        TBL1.date = Date_ADD(TBL2.date, interval 1 day) and
                        TBL1.time = '09:00:00'
                        )
                    )
                    
        """
    
        df_input_M = pd.read_sql(sql,db)
        # 일봉 데이터 가져오기
    
        time_list = [1,2,3,5,10]
        date_30B = date_list[date_index - 30] #30일전
    
        sql = f"""
            select 
                TBL1.C_code as C_code, 
                TBL1.priceS / @BPS as priceS, 
                TBL1.priceH / @BPS as priceH, 
                TBL1.priceL / @BPS as priceL, 
                TBL2.priceE / @BPS as priceE, 
                TBL1.trade / @Issued_shares as trade
                from (
                    select 
                        C_code, 
                        date, 
                        time, 
                        priceS, 
                        max(priceH) as priceH, 
                        min(priceL) as priceL, 
                        sum(trade) as trade 
                    from mdtbl2
                    where
                        C_code = '{code}'and
                        date < '{date}' and
                        date >= '{date_30B}'
                    group by C_code, date
                    ) as TBL1,
    
                    (
                    select C_code, date, priceE 
                    from mdtbl2
                    where 
                        C_code = '{code}'and
                        date < '{date}' and
                        date >= '{date_30B}' and
                        time = '15:30:00'
                    ) as TBL2
                where TBL1.C_code = TBL2.C_code and TBL1.date = TBL2.date
        """
        df_input_D = pd.read_sql(sql, db)
    
        #output 가져오기
        dates = [1,2,3,5,10,20]
        tmp_date = [date_list[date_index + x] for x in dates]
    
    
        sql = f"""
            select 
                (TBL1.priceH - @price) / @price_percent as AF_1M,
                (TBL2.priceH - @price) / @price_percent as AF_2M,
                (TBL3.priceH - @price) / @price_percent as AF_3M,
                (TBL4.priceH - @price) / @price_percent as AF_5M,
                (TBL5.priceH - @price) / @price_percent as AF_10M,
                (TBL6.priceH - @price) / @price_percent as Max_5M,
                (TBL7.priceH - @price) / @price_percent as Max_10M,
                (TBL8.priceH - @price) / @price_percent as Max_20M,
                (TBL9.priceH - @price) / @price_percent as Max_60M,
                (TBL10.priceH - @price) / @price_percent as Max_Today,
                (TBL11.priceH - @price) / @price_percent as Max_AF_1D,
                (TBL12.priceH - @price) / @price_percent as Max_AF_2D,
                (TBL13.priceH - @price) / @price_percent as Max_AF_3D,
                (TBL14.priceH - @price) / @price_percent as Max_AF_5D,
                (TBL15.priceH - @price) / @price_percent as Max_AF_10D,
                (TBL16.priceH - @price) / @price_percent as Max_3D,
                (TBL17.priceH - @price) / @price_percent as Max_5D,
                (TBL18.priceH - @price) / @price_percent as Max_10D,
                (TBL19.priceH - @price) / @price_percent as Max_20D
            from (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{date}' and 
                    time = Date_Add(time('{time}'), interval 1 minute)
            ) as TBL1,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{date}' and 
                    time = Date_Add(time('{time}'), interval 2 minute)
            ) as TBL2,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{date}' and 
                    time = Date_Add(time('{time}'), interval 3 minute)
            ) as TBL3,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{date}' and 
                    time = Date_Add(time('{time}'), interval 5 minute)
            ) as TBL4,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{date}' and 
                    time = Date_Add(time('{time}'), interval 10 minute)
            ) as TBL5,
    
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{date}' and 
                    time > '{time}' and
                    time <= Date_Add(time('{time}'), interval 5 minute)
            ) as TBL6,
    
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where
                    C_code = '{code}' and 
                    date = '{date}' and 
                    time > '{time}' and
                    time <= Date_Add(time('{time}'), interval 10 minute)
            ) as TBL7,
    
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{date}' and 
                    time > '{time}' and
                    time <= Date_Add(time('{time}'), interval 20 minute)
            ) as TBL8,
    
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{date}' and 
                    time > '{time}' and
                    time <= Date_Add(time('{time}'), interval 60 minute)
            ) as TBL9,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{date}' and 
                    time >= '{time}'
            ) as TBL10,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{tmp_date[0]}'
            ) as TBL11,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{tmp_date[2]}'
            ) as TBL12,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{tmp_date[3]}'
            ) as TBL13,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date = '{tmp_date[4]}'
            ) as TBL14,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date ='{tmp_date[5]}'
            ) as TBL15,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date > '{date}' and
                    date <= '{tmp_date[2]}'
            ) as TBL16,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date > '{date}' and
                    date <= '{tmp_date[3]}'
            ) as TBL17,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date > '{date}' and
                    date <= '{tmp_date[4]}'
            ) as TBL18,
            
            (
                select ifnull(max(priceH),0) as priceH
                from mdtbl2
                where 
                    C_code = '{code}' and 
                    date > '{date}' and
                    date <= '{tmp_date[5]}'
            ) as TBL19
        """
    
    
        df_output = pd.read_sql(sql, db)
    
        df_list = [df_input_M,df_input_D,df_output]
        return df_list
    except:
        pass
    finally:
        #print('걸린시간 : %s' %(dt.datetime.now() - start_time))
        db_index.append(index)
        progress[index] = progress[index] + 1
        my_pbar(sum(progress),len(input_data))

### Function **minute_frame**
* 매개변수(parameter) : df, column_num_M
> data_making으로 뽑아낸 데이터를 정규화 해주는 함수

In [ ]:
def minute_frame(df,column_num_M):
    ary = np.zeros((60,column_num_M), dtype = object)
    data_len = len(df)
    ary[60-data_len:,:] = df

    return ary

### Function **day_frame**
* 매개변수(parameter) : df, column_num_D
> data_making으로 뽑아낸 데이터를 정규화 해주는 함수

In [ ]:
def day_frame(df,column_num_D):
    ary = np.zeros((30,column_num_D), dtype = object)
    data_len = len(df)
    ary[30-data_len:,:] = df

    return ary

# 실제 데이터 추출

## input_data 추출
* multiprocessing을 이용하여 데이터를 빠르게 추출한다.

### process 만들기

In [ ]:
pool = Pool(num_cores)

### max_num까지 30만개 뽑기

In [ ]:
sql = '''
    select max(num) as num from mdtbl2
'''
max_num = pd.read_sql(sql,db)['num'][0]
num_list = random.sample(range(1,max_num),300000)

### input_data 추출하기

In [ ]:
start_time = dt.datetime.now()
progress = manager.list([0 for x in range(num_cores)])
input_data = pool.map(input_making, num_list)
my_pbar(100,100)
progress = manager.list([0 for x in range(num_cores)])
print('총 걸린시간 : %s' %(dt.datetime.now() - start_time))
total = len(input_data)
input_data = list(filter(None, input_data))
dummy = total - len(input_data)
print(f'None값을 가진 데이터 {dummy}개를 제외하고, 총 {len(input_data)}개의 데이터 추출')

#### date기준 필터링
* 현재까지 수집해둔 데이터(mysql에 들어있는)가 2020년 1월 ~ 2021년 1월 즈음으로, 모든 데이터를 이용하기에는 무리가 있어 해당 날짜의 input_data만 사용

In [13]:
start_date = dt.datetime.strptime('2020-02-20', '%Y-%m-%d')
end_date = dt.datetime.strptime('2020-11-20', '%Y-%m-%d')
input_data = [x for x in input_data if (dt.datetime.strptime(x[1], '%Y-%m-%d') >= start_date and dt.datetime.strptime(x[1], '%Y-%m-%d') <= end_date)]
print(f'현재 가진 데이터로는 추출 불가능한 데이터 {total - dummy - len(input_data)}개를 제외하고, 총 {len(input_data)}개의 데이터 추출')

pool.close()
pool.join()

NameError: ignored

## train_data 추출

In [ ]:
num_cores = mp.cpu_count()
 
pool = Pool(num_cores) # process 만들기
 
start_time = dt.datetime.now()
train_data = pool.starmap(data_making, input_data)
my_pbar(100,100)
train_data = list(filter(None, train_data))
print('총 걸린시간 : %s' %(dt.datetime.now() - start_time))
pool.close()
pool.join()

## train_data 정규화

In [ ]:
column_num_M = len(train_data[0][0].transpose())
column_num_D = len(train_data[0][1].transpose())

#60분 동안의 데이터를 수집할 것이기 떄문에, 60으로 설정

train_data_M = [minute_frame(x[0],column_num_M) for x in train_data]
train_data_D = [day_frame(x[1],column_num_D) for x in train_data]
train_data_Out = [x[2] for x in train_data]

train_ary_M = np.stack(train_data_M, axis = 0)
train_ary_D = np.stack(train_data_D, axis = 0)
train_ary_Out = np.stack(train_data_Out, axis = 0)



## numpy파일로 저장

In [ ]:
path = r'/content/gdrive/My Drive/인공지능 데이터/'

np.save(path + '60분데이터',train_ary_M)
np.save(path + '30일데이터',train_ary_D)
np.save(path + '결과데이터',train_ary_Out)

# dd